In [ ]:
import json

import pandas as pd
import requests

import my_config

class RestAPI:
    def __init__(self, base_url, endpoints):
        self.base_url = base_url
        self.endpoints = endpoints
        self.data = {}

    def fetch_data(self):
        for key, endpoint in self.endpoints.items():
            url = self.base_url + endpoint
            response = requests.get(url)
            if response.status_code == 200: 
                self.data[key] = response.json()
            else: 
                print(f"Error: {response.status_code}")

In [ ]:
dates = [2018, 2022]
selected_treasury_tables = ['v1/accounting/dts/dts_table_1']

from src.callbacks.bea_fred_api import RestAPI, add_date_table, create_date_series

table_columns = {
    'v1/accounting/dts/dts_table_1': ['close_today_bal', 'account_type'],
    'v2/accounting/od/debt_to_penny': ['tot_pub_debt_out_amt']
}

table_column_names = {
    'v1/accounting/dts/dts_table_1': {'close_today_bal': 'closing_daily_balance'},
    'v2/accounting/od/debt_to_penny': {'tot_pub_debt_out_amt': 'Total_Outstanding_Debt'}
}

class DataFetcher:
    @staticmethod
    def fetch_treasury_data(selected_treasury_tables, dates):
        treasury_base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/"

        treasury_endpoints = {
            f'{table}': f"{table}?&filter=record_fiscal_year:in:({dates})&page[size]=10000" for table in selected_treasury_tables
        }

        treasury_api = RestAPI(treasury_base_url, treasury_endpoints)
        treasury_api.fetch_data()
        return treasury_api
    
def process_treasury_table(api, table, table_columns, table_column_names):
    try:
        data = api.data[table]['data']
        df = pd.DataFrame(data, columns=table_columns[table]).rename(columns=table_column_names[table])

        if table == 'v1/accounting/dts/dts_table_1':
            df = df.loc[df['account_type'] == 'Federal Reserve Account'].drop(columns=['account_type']).reset_index(drop=True)
    except KeyError:
        print("error")
    return df

all_years_string = ','.join(str(year) for year in range(dates[0], dates[1] + 1))

date_table_added = add_date_table('v1/accounting/dts/dts_table_1', selected_treasury_tables)

treasury_api = DataFetcher.fetch_treasury_data(selected_treasury_tables, all_years_string)
date_sr = create_date_series('record_date', treasury_api.data['v1/accounting/dts/dts_table_1']['data'])

if date_table_added: selected_treasury_tables.remove('v1/accounting/dts/dts_table_1')

all_dfs = [date_sr]

for table in selected_treasury_tables:
    treasury_df = process_treasury_table(treasury_api, table, table_columns, table_column_names)
    if treasury_df is not None: all_dfs.append(treasury_df)

table_df = pd.concat(all_dfs , axis=1)

print(table_df.to_string())
print(dates)
dates_tuple = (dates[0], dates[1])
print(dates_tuple)

US Bureau of Economic Analysis API

In [ ]:
# catalogued BEA tables
all_bea_tables = {
    "T10503": "Real Gross Domestic Product, Expanded Detail, Quantity Indexes (A) (Q)",
    "T10105": "Gross Domestic Product (A) (Q)",
    "T10505": "Gross Domestic Product, Expanded Detail (A) (Q)",
    "T80103": "Real Gross Domestic Product, Quantity Indexes, Not Seasonally Adjusted (Q)",
    "T80104": "Price Indexes for Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80105": "Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80106": "Real Gross Domestic Product, Chained Dollars, Not Seasonally Adjusted (Q)",
    "T80200": "Gross Domestic Income by Type of Income, Not Seasonally Adjusted (Q)",
    "T11000": "Gross Domestic Income by Type of Income (A) (Q)",
    "T11000": "Gross Domestic Income by Type of Income (A) (Q)",

    "T20301": "Percent Change From Preceding Period in Real Personal Consumption Expenditures by Major Type of Product (A) (Q)",
    "T20303": "Real Personal Consumption Expenditures by Major Type of Product, Quantity Indexes (A) (Q)",
    "T20304": "Price Indexes for Personal Consumption Expenditures by Major Type of Product (A) (Q)",
    "T20305": "Personal Consumption Expenditures by Major Type of Product (A) (Q)",
    "T20306": "Real Personal Consumption Expenditures by Major Type of Product, Chained Dollars (A) (Q)",
    "T20307": "Percent Change From Preceding Period in Prices for Personal Consumption Expenditures by Major Type of Product (A) (Q)",

    "T20100": "Personal Income and Its Disposition (A) (Q)",    
    "T20600": "Personal Income and Its Disposition, Monthly (M)",
    "T20200A": "Wages and Salaries by Industry (A) (Q)",
    "T20200B": "Wages and Salaries by Industry (A) (Q)",
    "T20700A": "Wages and Salaries by Industry, Monthly (M)",
    "T20700B": "Wages and Salaries by Industry, Monthly (M)",

    "T40201": "Percent Change From Preceding Period in Real Exports and in Real Imports of Goods and Services by Type of Product (A) (Q)",
    "T40203": "Real Exports and Imports of Goods and Services by Type of Product, Quantity Indexes (A) (Q)",
    "T40204": "Price Indexes for Exports and Imports of Goods and Services by Type of Product (A) (Q)",
    "T40205": "Exports and Imports of Goods and Services by Type of Product (A) (Q)",

    "T50100": "Saving and Investment by Sector (A) (Q)",
    "T50203": "Real Gross and Net Domestic Investment by Major Type, Quantity Indexes (A)",
    "T50205": "Gross and Net Domestic Investment by Major Type (A)",
    "T50301": "Percent Change From Preceding Period in Real Private Fixed Investment by Type (A) (Q)",

    "T50705A": "Change in Private Inventories by Industry (A) (Q)",
    "T50705B": "Change in Private Inventories by Industry (A) (Q)",

    "T61600A": "Corporate Profits by Industry (A)",
    "T61600B": "Corporate Profits by Industry (A) (Q)",
    "T61600C": "Corporate Profits by Industry (A) (Q)",
    "T61600D": "Corporate Profits by Industry (A) (Q)",

    "T70201A": "Percent Change From Preceding Period in Real Auto Output (A) (Q)",
    "T70201B": "Percent Change From Preceding Period in Real Motor Vehicle Output (A) (Q)",
    "T70203A": "Real Auto Output, Quantity Indexes (A) (Q)",
    "T70203B": "Real Motor Vehicle Output, Quantity Indexes (A) (Q)",
    "T70204A": "Price Indexes for Auto Output (A) (Q)",
    "T70204B": "Price Indexes for Motor Vehicle Output (A) (Q)",
    "T70205A": "Auto Output (A) (Q)",
    "T70205B": "Motor Vehicle Output (A) (Q)",

    "T80300": "Federal Government Current Receipts and Expenditures, Not Seasonally Adjusted (Q)"
}

gdp_percent_changes = {
    "T10101" : "Percent Change From Preceding Period in Real Gross Domestic Product (A) (Q)",
    "T10107": "Percent Change From Preceding Period in Prices for Gross Domestic Product (A) (Q)",
    "T10108": "Contributions to Percent Change in the Gross Domestic Product Price Index (A) (Q)",
    "T10501": "Percent Change From Preceding Period in Real Gross Domestic Product, Expanded Detail (A) (Q)",
    "T80111": "Real Gross Domestic Product: Percent Change From Quarter One Year Ago, Not Seasonally Adjusted (Q)",
    "T80103": "Real Gross Domestic Product, Quantity Indexes, Not Seasonally Adjusted (Q)",
    "T80104": "Price Indexes for Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80105": "Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80106": "Real Gross Domestic Product, Chained Dollars, Not Seasonally Adjusted (Q)",
}

In [ ]:
# Checking BEA table names
bea_base_url = f"https://apps.bea.gov/api/data/"

bea_all_endpoints = {
    'all_nipa': f"?&UserID={my_config.BEA_KEY}&DataSetName=NIPA&method=GetParameterValues&ParameterName=TableName",
    'all_fixedassets': f"?&UserID={my_config.BEA_KEY}&method=GetParameterValues&DataSetName=FixedAssets&ParameterName=TableName",
    'all_gdpindustry': f"?&UserID={my_config.BEA_KEY}&method=GetParameterValues&DataSetName=GDPbyIndustry&ParameterName=TableID",
    'all_io': f"?&UserID={my_config.BEA_KEY}&method=GetParameterValues&DataSetName=InputOutput&ParameterName=TableID"
}

bea_api = RestAPI(bea_base_url, bea_all_endpoints)
bea_api.fetch_data()

print(json.dumps(bea_api.data['all_io']['BEAAPI']['Results'], indent=4))

In [ ]:
# Checking table metrics and measures
start_year = 2019
end_year = 2020
bea_years = range(start_year, end_year + 1)
bea_years = ','.join(str(year) for year in bea_years)

bea_frequency = 'Q'
bea_dataset = 'NIPA'
selected_tables = ['T20304']

bea_base_url = f"https://apps.bea.gov/api/data/"

bea_endpoints = {
    f'{table}': (f"?&UserID={my_config.BEA_KEY}"
                 f"&method=GetData"
                 f"&DataSetName={bea_dataset}"
                 f"&TableName={table}"
                 f"&Frequency={bea_frequency}"
                 f"&Year={bea_years}")
    for table in selected_tables
}

bea_api = RestAPI(bea_base_url, bea_endpoints)
bea_api.fetch_data()

print(json.dumps(bea_api.data['T20304']['BEAAPI']['Results']['Data'], indent=4))

.py file block

In [ ]:
start_year = 2020
end_year = 2022
selected_bea_tables = ['T20307']
selected_fred_tables = ['CPIAUCSL']
selected_treasury_tables = []

all_years_string = range(start_year, end_year + 1)
all_years_string = ','.join(str(year) for year in all_years_string)

T10101_added = 'T10101' not in selected_bea_tables
if T10101_added: selected_bea_tables += ['T10101']

bea_base_url = f"https://apps.bea.gov/api/data/"

# The multiselect components' value: datasets, is a list of table names.
# We can use dictionary comprehension to create the endpoints from this, with the table names are used as the keys.
bea_endpoints = {
    f'{table}': (f"?&UserID={my_config.BEA_KEY}"
                f"&method=GetData"
                f"&DataSetName=NIPA"
                f"&Frequency=Q"
                f"&TableName={table}"
                f"&Year={all_years_string}")
    for table in selected_bea_tables
}

# In general, each table contains multiple economic metrics, for example, gross domestic product and gross national product.
# We can use a dictionary to filter on the metric we're interested in per table rather than doing several if statements.
filter_metrics = {
    "T10101": "Gross domestic product",
    'T10105': "Gross domestic product",
    'T10107': "Gross domestic product",
    'T20100': "Personal income",
    'T20200B': "Wages and salaries",
    'T20307': "Personal consumption expenditures (PCE)"
}

def create_date_index(date_name, data):
    periods = [item[date_name] for item in data]
    unique_periods = list(set(periods))
    date_sr = pd.Series(unique_periods, name='date').sort_values().reset_index(drop=True)
    return date_sr

bea_api = RestAPI(bea_base_url, bea_endpoints)
bea_api.fetch_data()

date_sr = create_date_index('TimePeriod', bea_api.data['T10101']['BEAAPI']['Results']['Data'])

if T10101_added: selected_bea_tables.remove('T10101')

# Replacing the JSON data within the bea_api.data dictionary with formatted DataFrames.
all_dfs = [date_sr]
for table in selected_bea_tables:
    try:
        bea_api.data[table] = (
            pd.DataFrame(bea_api.data[table]['BEAAPI']['Results']['Data'], columns=['DataValue', 'METRIC_NAME', 'LineDescription']) \
            .rename(columns={'DataValue': table + '_value', 'METRIC_NAME': table + '_metric'}))
    except KeyError:
        print("Invalid Years: No data is available within the selected years for one of the requested datasets.")
    
    if table != 'T20307':
        bea_api.data[table] = (
            bea_api.data[table].loc[bea_api.data[table]['LineDescription'] == filter_metrics[table]]
            .drop(columns=['LineDescription']))

    # The tables contain multiple values for a given economic measure using differnt methodologies. These are vertically stacked.
    # To avoid needing to filter on the method column, the table is pivotted in a way that doesn't produce NaN values:
    sliced_dfs = []
    unique_metrics = bea_api.data[table][table + '_metric'].unique()
    for metric in unique_metrics:
        sliced_df = (
            bea_api.data[table].loc[bea_api.data[table][table + '_metric'] == metric]
            .drop(columns=[table + '_metric'])
            .reset_index(drop=True)
            .rename(columns={f'{table}_value': f'{table} - ' + metric})
        )
        sliced_dfs.append(sliced_df)
    merged_df = pd.concat(sliced_dfs, axis=1)
    all_dfs.append(merged_df)

fred_units = "pch"
fred_frequency = "q"
fred_start_year = f"{start_year}-01-01"
fred_end_year = f"{end_year}-01-01"

fred_base_url = "https://api.stlouisfed.org/fred/series/observations"

fred_endpoints = {
    f'{table}': (f"?series_id={table}"
                    f"&api_key={my_config.FRED_KEY}"
                    f"&file_type=json"
                    f"&observation_start={fred_start_year}"
                    f"&observation_end={fred_end_year}"
                    f"&units={fred_units}"
                    f"&frequency={fred_frequency}"
                    f"&aggregation_method=sum")
    for table in selected_fred_tables
}

fred_api = RestAPI(fred_base_url, fred_endpoints)
fred_api.fetch_data()

for table in selected_fred_tables:
    try:
        fred_df = (
            pd.DataFrame(fred_api.data[table]['observations'], columns=['value']) \
            .rename(columns={'value': table + '_value'}))
    except KeyError:
        print("There was a problem requesting the selected data from FRED.")
    all_dfs.append(fred_df)

table_df = pd.concat(all_dfs , axis=1)

print(table_df)

US Treasury Department API

In [7]:
'''
Treasury Endpoint Descriptions:
"interest_rates" - Average interest rates for marketable and non-marketable securities.
"cash_balance" - This table represents the Treasury General Account balance. All figures are rounded to the nearest million.
"transactions" - This table represents deposits and withdrawals from the Treasury General Account. 
"us_debt" - Outstanding U.S. debt on a daily basis. Measured to the penny.
All figures are rounded to the nearest million.
'''

# 'interest_rates': 'v2/accounting/od/avg_interest_rates',
# 'cash_balance': 'v1/accounting/dts/dts_table_1',
# 'transactions': 'v1/accounting/dts/dts_table_2',
# 'us_debt': 'v2/accounting/od/debt_to_penny',
# 'balance_sheet': 'v2/accounting/od/balance_sheets'

start_year = 2016
end_year = 2017
selected_treasury_tables = ['v2/accounting/od/avg_interest_rates']

all_years_string = range(start_year, end_year + 1)
all_years_string = ','.join(str(year) for year in all_years_string)

treasury_base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/"
treasury_endpoints = {
    f'{table}': f"{table}?&filter=record_fiscal_year:in:({all_years_string})" for table in selected_treasury_tables
}

treasury_api = RestAPI(treasury_base_url, treasury_endpoints)
treasury_api.fetch_data()

print(json.dumps(treasury_api.data['v2/accounting/od/avg_interest_rates']['data'], indent=4))

# # Cash balance
# for table in selected_treasury_tables:
#     try:
#         treasury_api.data[table] = (
#             pd.DataFrame(treasury_api.data[table]['data'], columns=['record_date', 'close_today_bal', 'account_type', 'tot_pub_debt_out_amt']) \
#             .rename(columns={'close_today_bal': 'closing_daily_balance'}))
#     except KeyError:
#         print('Invalid Years: No data is available within the selected years for one of the requested datasets.')

#     # treasury_api.data[table] = (
#     #     treasury_api.data[table].loc[treasury_api.data[table]['account_type'] == 'Federal Reserve Account']
#     #     .drop(columns=['account_type'])).reset_index(drop=True)

# print(treasury_api.data['v2/accounting/od/debt_to_penny'])

[
    {
        "record_date": "2015-10-31",
        "security_type_desc": "Marketable",
        "security_desc": "Treasury Bills",
        "avg_interest_rate_amt": "0.144",
        "src_line_nbr": "1",
        "record_fiscal_year": "2016",
        "record_fiscal_quarter": "1",
        "record_calendar_year": "2015",
        "record_calendar_quarter": "4",
        "record_calendar_month": "10",
        "record_calendar_day": "31"
    },
    {
        "record_date": "2015-10-31",
        "security_type_desc": "Marketable",
        "security_desc": "Treasury Notes",
        "avg_interest_rate_amt": "1.834",
        "src_line_nbr": "2",
        "record_fiscal_year": "2016",
        "record_fiscal_quarter": "1",
        "record_calendar_year": "2015",
        "record_calendar_quarter": "4",
        "record_calendar_month": "10",
        "record_calendar_day": "31"
    },
    {
        "record_date": "2015-10-31",
        "security_type_desc": "Marketable",
        "security_desc": "Treasur

In [ ]:
# Depostits and Withdraws
start_year = 2017
end_year = 2023
selected_treasury_tables = ['v1/accounting/dts/dts_table_2']

all_years_string = range(start_year, end_year + 1)
all_years_string = ','.join(str(year) for year in all_years_string)

treasury_base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/"
treasury_endpoints = {
    f'{table}': f"{table}?&filter=record_fiscal_year:in:({all_years_string})" for table in selected_treasury_tables
}

treasury_api = RestAPI(treasury_base_url, treasury_endpoints)
treasury_api.fetch_data()

print(json.dumps(treasury_api.data['v1/accounting/dts/dts_table_2']['data'], indent=4))

for table in selected_treasury_tables:
    try:
        treasury_api.data[table] = (
            pd.DataFrame(treasury_api.data[table]['data'], columns=['record_date', 'transaction_type', 'transaction_today_amt']) \
            .rename(columns={'transaction_today_amt': 'amount ($M)'}))
        treasury_api.data[table] = treasury_api.data[table].astype({"transaction_type": str, "amount ($M)": float})
    except KeyError:
        print('Invalid Years: No data is available within the selected years for one of the requested datasets.')

    treasury_api.data[table] = (
        treasury_api.data[table]
        .groupby(['record_date', 'transaction_type'])['amount ($M)']
        .sum()
        .reset_index()
        .drop(columns=['record_date'])
    )

    sliced_dfs = []
    unique_metrics = treasury_api.data[table]['transaction_type'].unique()
    for metric in unique_metrics:
        sliced_df = (
            treasury_api.data[table].loc[treasury_api.data[table]['transaction_type'] == metric]
            .drop(columns=['transaction_type'])
            .reset_index(drop=True)
            .rename(columns={'amount ($M)': f'{metric} ' + 'amount ($M)'})
        )
        sliced_dfs.append(sliced_df)
    merged_df = pd.concat(sliced_dfs, axis=1)

    print(merged_df)

In [ ]:
# US Debt
start_year = 2017
end_year = 2023
selected_treasury_tables = ['v2/accounting/od/debt_to_penny']

all_years_string = range(start_year, end_year + 1)
all_years_string = ','.join(str(year) for year in all_years_string)

treasury_base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/"
treasury_endpoints = {
    f'{table}': f"{table}?&filter=record_fiscal_year:in:({all_years_string})" for table in selected_treasury_tables
}

treasury_api = RestAPI(treasury_base_url, treasury_endpoints)
treasury_api.fetch_data()

print(json.dumps(treasury_api.data['v2/accounting/od/debt_to_penny']['data'], indent=4))

for table in selected_treasury_tables:
    try:
        treasury_api.data[table] = (
            pd.DataFrame(treasury_api.data[table]['data'], columns=['record_date', 'transaction_type', 'transaction_today_amt']) \
            .rename(columns={'transaction_today_amt': 'amount ($M)'}))
        treasury_api.data[table] = treasury_api.data[table].astype({"transaction_type": str, "amount ($M)": float})
    except KeyError:
        print('Invalid Years: No data is available within the selected years for one of the requested datasets.')

    treasury_api.data[table] = (
        treasury_api.data[table]
        .groupby(['record_date', 'transaction_type'])['amount ($M)']
        .sum()
        .reset_index()
        # .drop(columns=['record_date'])
    )

    sliced_dfs = []
    unique_metrics = treasury_api.data[table]['transaction_type'].unique()
    for metric in unique_metrics:
        sliced_df = (
            treasury_api.data[table].loc[treasury_api.data[table]['transaction_type'] == metric]
            .drop(columns=['transaction_type'])
            .reset_index(drop=True)
            .rename(columns={'amount ($M)': f'{metric} ' + 'amount ($M)'})
        )
        sliced_dfs.append(sliced_df)
    merged_df = pd.concat(sliced_dfs, axis=1)

    print(merged_df)

In [ ]:
start_year = 2016
end_year = 2022
selected_treasury_tables = ['v2/accounting/od/debt_to_penny']
# selected_treasury_tables = ['v1/accounting/dts/dts_table_1']

all_years_string = range(start_year, end_year + 1)
all_years_string = ','.join(str(year) for year in all_years_string)

if 'v1/accounting/dts/dts_table_1' not in selected_treasury_tables: 
    selected_treasury_tables += ['v1/accounting/dts/dts_table_1']
    dts_table_1_added = True
else: 
    dts_table_1_added = False

treasury_base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/"
treasury_endpoints = {
    f'{table}': f"{table}?&filter=record_fiscal_year:in:({all_years_string})" for table in selected_treasury_tables
}

treasury_api = RestAPI(treasury_base_url, treasury_endpoints)
treasury_api.fetch_data()

print(json.dumps(treasury_api.data['v2/accounting/od/debt_to_penny']['data'], indent=4))
# print(json.dumps(treasury_api.data['v1/accounting/dts/dts_table_1']['data'], indent=4))

# Creating a date series to index the user selected DataFrames.
periods = [item['record_date'] for item in treasury_api.data['v1/accounting/dts/dts_table_1']['data']]
unique_periods = list(set(periods))
date_sr = pd.Series(unique_periods, name='date').sort_values().reset_index(drop=True)

if dts_table_1_added: selected_treasury_tables.remove('v1/accounting/dts/dts_table_1')

# Replacing the JSON data within the bea_api.data dictionary with formatted DataFrames.
all_dfs = [date_sr]
for table in selected_treasury_tables:
    if table == 'v1/accounting/dts/dts_table_1':
        try:
            treasury_api.data[table] = (
                pd.DataFrame(treasury_api.data[table]['data'], columns=['close_today_bal', 'account_type']) \
                .rename(columns={'close_today_bal': 'closing_daily_balance'}))
        except KeyError:
            print("Error")

        treasury_api.data[table] = (
            treasury_api.data[table].loc[treasury_api.data[table]['account_type'] == 'Federal Reserve Account']
            .drop(columns=['account_type'])).reset_index(drop=True)
    elif table == 'v2/accounting/od/debt_to_penny':
        try:
            treasury_api.data[table] = (
                pd.DataFrame(treasury_api.data[table]['data'], columns=['tot_pub_debt_out_amt']) \
                .rename(columns={'tot_pub_debt_out_amt': 'Total_Outstanding_Debt'}))
        except KeyError:
            print("Error")

    all_dfs.append(treasury_api.data[table])
table_df = pd.concat(all_dfs , axis=1)

print(table_df)

FRED API

In [ ]:
fred_units = "pch"
fred_frequency = "q"
fred_start_date = "2005-01-01"
fred_end_date = "2020-01-01"
selected_fred_tables = ["CPIAUCSL"]

fred_base_url = "https://api.stlouisfed.org/fred/series/observations"

fred_endpoints = {
    f'{table}': (f"?series_id={table}"
                      f"&api_key={my_config.FRED_KEY}"
                      f"&file_type=json"
                      f"&observation_start={fred_start_date}"
                      f"&observation_end={fred_end_date}"
                      f"&units={fred_units}"
                      f"&frequency={fred_frequency}"
                      f"&aggregation_method=sum")
    for table in selected_fred_tables
}
 
fred_api = RestAPI(fred_base_url, fred_endpoints)
fred_api.fetch_data()

print(json.dumps(fred_api.data['CPIAUCSL'], indent=4))

# for table in selected_fred_tables:
#     try:
#         fred_api.data[table] = (
#             pd.DataFrame(fred_api.data[table]['observations'], columns=['value']) \
#             .rename(columns={'value': table + '_value'}))
#     except KeyError:
#         print("Failed formatting")

# print(fred_api.data['CPIAUCSL'].head())

US Bureau of Labor Statistics API - really slow

In [ ]:
labor_base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

labor_endpoints = {
    'unemployment_rate': 'LNS14000000',
    # 'cpi': 'CUUR0000SA0'
}

labor_dates = {
    'startyear': '2005',
    'endyear': '2020'
}

labor_api = RestAPI(labor_base_url, labor_endpoints)
labor_api.fetch_data()

print(json.dumps(labor_api.data['unemployment_rate'], indent=4))

In [ ]:
# Save json output to a .json file.
json_api_2 = json.dumps(bea_api.data['annual_gdp']['BEAAPI']['Results']['Data'], indent=4)
func = open("dict_2.json","w")
func.write(json_api_2)
func.close()

In [ ]:
def add_date_table(table, dataset_list):
    date_table_added = table not in dataset_list
    if date_table_added: dataset_list += [table]
    return date_table_added

def create_date_series(date_name, data):
    periods = [item[date_name] for item in data]
    unique_periods = list(set(periods))
    date_sr = pd.Series(unique_periods, name='date').sort_values().reset_index(drop=True)
    return date_sr